In [37]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
# !pip install finance-datareader
import FinanceDataReader as fdr
import numpy as np
import os

# Request 

In [247]:
r = requests.get("https://www.wooribank.com/",
                  timeout = 1,
                  params = {
                      "User-agent" : "~~"
                  }
                ) 
print(r.status_code)

print(r.encoding)

# encoding 방식 지정
r.encoding = "utf-8"
# print(r.encoding)

print(r.text)

200
ISO-8859-1
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
    
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">
<head>
	<meta http-equiv="x-ua-compatible" content="IE=edge">
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<meta name="naver-site-verification" content="a8f714bdbbf08acd830048b83dcdb1078a4aeb9b" />
	<meta name="description" content="오늘의 혁신으로 내일의 가치를 만드는 우리은행 공식 사이트입니다">
	<meta name="keywords" content="우리은행, 우리은행 인터넷뱅킹, 계좌조회, 이체, 대출, 외화 환전, 공과금납부, 상품신규, 사고신고 안내">
	<meta name="robots" content="index,follow">
	<title>우리은행</title>
	<link rel="canonical" href="http://www.wooribank.com">
	<link  rel="stylesheet" href="https://simg.wooribank.com/css/base.css?1547196841000" type="text/css" />

	<link  rel="stylesheet" href="https://simg.wooribank.com/css/intro_new.css?1638342698000" type="t

# BeautifulSoup

In [237]:
soup = BeautifulSoup(r.content, 'html.parser')
# print(soup.prettify())

print(soup.title)
print(soup.title.string) 

<title>우리은행</title>
우리은행


In [238]:
for item in soup.select("div[class='tit-5 absolute js-display-hover'] li"):
    print(item.text)
    

금융상품
추천상품
예금
펀드
대출
외환
골드
신탁
보험
퇴직연금
시니어플러스 은퇴설계
ISA


# Request, BS4 실습

In [250]:
# 오늘의 코스피 받아오기
r = requests.get("https://finance.naver.com/sise/sise_index.naver?code=KOSPI")
print(r.encoding)
soup = BeautifulSoup(r.text, 'html.parser')

tag = soup.select("em[id='now_value']" )[0]

tag.text

EUC-KR


'2,729.98'

In [251]:
# 시간별 날씨
r = requests.get("https://weather.naver.com/today/")
print(r.encoding)
print(r.content)

UTF-8
b'<!doctype html>\n<html lang="ko" data-platform="pc" data-useragent="python-requests/2.27.1">\n    <head>\n        <meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=device-width,initial-scale=1.0,user-scalable=no,maximum-scale=1">\n<meta name="format-detection" content="telephone=no">\n<meta property="og:title" content="\xeb\x84\xa4\xec\x9d\xb4\xeb\xb2\x84 \xeb\x82\xa0\xec\x94\xa8"/>\n<meta property="og:url" content="http://weather.naver.com/today/"/>\n<meta property="og:image" content="https://ssl.pstatic.net/static/weather/m/og_weather.png"/>\n<meta property="og:description" content="\xea\xb5\xad\xeb\x82\xb4\xec\x99\xb8 \xeb\x82\xa0\xec\x94\xa8\xec\x99\x80 \xeb\xaf\xb8\xec\x84\xb8\xeb\xa8\xbc\xec\xa7\x80\xec\x97\x90 \xeb\x8c\x80\xed\x95\x9c \xec\xa2\x85\xed\x95\xa9\xec\xa0\x95\xeb\xb3\xb4 \xec\xa0\x9c\xea\xb3\xb5"/>\n<link rel="shortcut icon" href="https://ssl.pstatic.net/static/www/nFavicon24.png" type="image/x

In [252]:
soup = BeautifulSoup(r.content, 'html.parser')

In [253]:
# 시간대별 날씨 탭
# 마우스 우클릭 -> 페이지 소스 보기 or 검사 
# html 코드에서 시간대별 날씨 데이터가 포함된 태그 

# <div class="scroll_area" data-name="hourly" data-click="false" data-nclk="wtk.hourscr">
#   <div>
#     <ul>
#       <li>
#         <span class="time">16시</span>
#         <span class="blind">흐림</span>
#         <span class="blind">17도</span>
#       </li>
#     </ul>
#   </div>
# </div>

# data-name 이 hourly인 div 태그로부터 시작 !

# soup.select("")

hourly_weather = {}
for k, weather in enumerate(soup.select("div[data-name='hourly'] div ul li span" )):
    if k%3==0:
        time = weather.text
    if k%3==2:
        temparature = weather.text
        hourly_weather[time] = temparature
print(hourly_weather)

{'17시': '10도', '18시': '9도', '19시': '8도', '20시': '7도', '21시': '6도', '22시': '5도', '23시': '5도', '내일': '14도', '1시': '7도', '2시': '6도', '3시': '6도', '4시': '5도', '5시': '5도', '6시': '5도', '7시': '5도', '8시': '6도', '9시': '8도', '10시': '9도', '11시': '10도', '12시': '11도', '13시': '11도', '14시': '11도', '15시': '11도', '16시': '11도', '모레': '8도', '글피': '4도'}


# 투자지표 수집

In [2]:
stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체
stocks.dropna(inplace = True) # 지표가 없는 선물 등은 홈페이지, 지역, 산업 등이 모두 결측으로 되어 있으므로 삭제하면 됨
base_url = "https://search.itooza.com/search.htm?seName={}&jl=&search_ck=&sm=&sd=2021-08-09&ed=2021-09-08&ds_de=&page=&cpv="

In [3]:
for code, name in stocks[:20][['Symbol', 'Name']].values:
    print(name)
    while True:
        try:
            url = base_url.format(code)
            html = requests.get(url, headers={'User-Agent':'Gils'}).content
            soup = BeautifulSoup(html, 'lxml')
            break
        except:
            time.sleep(15 * 60)
    
    # 만약 해당 종목 자료가 없다면, 아래 테이블이 없음
    table = soup.find("table", {"class":"ex"}) # class가 ex인 테이블을 찾음
    
    if table == None: # class가 ex인 테이블이 없으면 크롤링하지 않음
        continue

    idx = ["2021-2Q", "2021-1Q",
           "2020-4Q", "2020-3Q", "2020-2Q", "2020-1Q",
           "2019-4Q", "2019-3Q", "2019-2Q", "2019-1Q",
           "2018-4Q", "2018-3Q"] # 인덱스를 직접 정의

    table_body = table.find('tbody')  # table에서 tbody만 가져옴
    tr_list = table_body.find_all('tr') # tbody에서 행을 다 찾아서 저장

    result_dict = dict()
    for tr in tr_list:
        col_name = tr.find('th').text # 투자 지표 이름
        values = [td.text for td in tr.find_all('td')] # tr에 있는 모든 td의 text만 가져와서 저장
        result_dict[col_name] = values # 사전으로 저장

    result = pd.DataFrame.from_dict(result_dict)
    result.drop('주가', axis = 1, inplace = True) # 주가열도 필요없으니 삭제
    result['분기'] = idx
    
    result.to_csv(name + ".csv", index = False, encoding = "cp949")
    
    time.sleep(1) # 1초씩 재우기

3S
AJ네트웍스
AK홀딩스
APS홀딩스
AP시스템
AP위성
BGF
BGF리테일
BNGT
BNK금융지주
BYC
CBI
CJ
CJ CGV
CJ ENM
CJ 바이오사이언스
CJ대한통운
CJ씨푸드
CJ제일제당
CJ프레시웨이


# 한경컨센서스 리포트 수집

In [4]:
import requests
import re

In [5]:
stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체

In [6]:
stocks['Symbol'] = stocks['Symbol'].astype(str)
def remove_noise_and_split_title(title):
    in_code = ''
    in_name = ''
    
    for code, name in stocks[['Symbol', 'Name']].values:
        if code in title and name in title:
            in_code = code
            in_name = name
    
    # 한글, 영어, 숫자 외 노이즈 제거
    clean_title = re.sub('[^A-Za-z0-9가-힣]', ' ', title)
    
    # 기업명 코드 수정
    clean_title = clean_title.replace(in_code, ' ')
    clean_title = clean_title.replace(in_name, ' ')
    while ' ' * 2 in clean_title:
        clean_title = clean_title.replace(' ' * 2, ' ')
    
    if in_name == '': # 기업명이 없는 제목이라면, 데이터에 추가하지 않음
        return [None] 
    else:
        return [in_name, in_code, clean_title]  

In [7]:
base_url = "http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate=2016-09-01&edate=2021-08-31&report_type=CO&order_type=&now_page={}"
data = [] 

for page_no in range(1, 10):
    while True:
        try:
            url = base_url.format(page_no)
            html = requests.get(url, headers={'User-Agent':'Gils'}).content
            soup = BeautifulSoup(html, 'lxml')
            print("{}/{}".format(page_no, 4284))
            break
        except:
            time.sleep(15 * 60)

    table = soup.find("div", {"class":"table_style01"}).find('table')
    for tr in table.find_all("tr")[1:]: # 1번째 행부터 순회
        record = []
        for i, td in enumerate(tr.find_all("td")[:6]): # 6번째 셀까지 순회
            if i == 1:
                record += remove_noise_and_split_title(td.text) # remove_noise_title의 출력과 이어 붙임
            elif i == 3: # 노이즈가 껴있는 세번째 셀만 따로 처리
                record.append(td.text.replace(" ", "").replace("\r","").replace("\n",""))
            else: # 1번째 셀이 아니면:
                record.append(td.text) # 셀의 텍스트 값 그대로 입력
            
        if None not in record: # 레코드에 None이 없으면
            data.append(record)

    time.sleep(1) # 연결 끊김 방지를 위해, 1초씩 재움
    
data = pd.DataFrame(data, columns = ["작성일", "종목명", "종목코드", "제목", "적정가격", "평가의견", "작성자", "작성기관"])
data.to_csv("리포트_데이터.csv", index = False, encoding = "cp949")

1/4284
2/4284
3/4284
4/4284
5/4284
6/4284
7/4284
8/4284
9/4284


# OpenDart 활용

In [10]:
import OpenDartReader

API 키 발급 링크 : https://opendart.fss.or.kr/uss/umt/EgovMberInsertView.do

In [11]:
my_api = "7c21274166c816929c5a5c6dacffaa8315b7daeb"
dart = OpenDartReader(my_api)

In [12]:
display(dart.report("SK하이닉스","배당",2020,"11011"))

,rcept_no,corp_cls,corp_code,corp_name,se,thstrm,frmtrm,lwfr
0,20210330000776,Y,00164779,SK하이닉스,주당액면가액(원),"5,000","5,000","5,000"
1,20210330000776,Y,00164779,SK하이닉스,(연결)당기순이익(백만원),"4,755,102","2,005,975","15,540,111"
2,20210330000776,Y,00164779,SK하이닉스,(별도)당기순이익(백만원),"4,217,841","1,476,981","15,407,086"
3,20210330000776,Y,00164779,SK하이닉스,(연결)주당순이익(원),"6,952","2,933","22,255"
4,20210330000776,Y,00164779,SK하이닉스,현금배당금총액(백만원),"800,282","684,002","1,026,003"
5,20210330000776,Y,00164779,SK하이닉스,주식배당금총액(백만원),-,-,-
6,20210330000776,Y,00164779,SK하이닉스,(연결)현금배당성향(%),16.80,34.10,6.60
7,20210330000776,Y,00164779,SK하이닉스,현금배당수익률(%),1.00,1.10,2.50
8,20210330000776,Y,00164779,SK하이닉스,현금배당수익률(%),-,-,-
9,20210330000776,Y,00164779,SK하이닉스,주식배당수익률(%),-,-,-


In [13]:
stock_list = fdr.StockListing("KRX")

In [14]:
stock_list = stock_list.loc[stock_list['Market'].isin(["KOSPI","KOSDAQ"])]
stock_name_list = stock_list.loc[stock_list['Region'].notnull(),'Name'].tolist()

In [15]:
def find_dividends(stock_name, year):
  stock_name_report = dart.report(stock_name, "배당", year, "11011")
  if stock_name_report is None:
    return np.nan, np.nan, np.nan
  else:
    
    stock_name_report = stock_name_report.loc[stock_name_report['se']=="주당 현금배당금(원)"].iloc[0]
    
    thstrm_dividends = int(stock_name_report['thstrm'].replace('-','0').replace(',',''))
    frmtrm_dividends = int(stock_name_report['frmtrm'].replace('-','0').replace(',',''))
    lwfr_dividends = int(stock_name_report['lwfr'].replace('-','0').replace(',',''))
    
    return lwfr_dividends, frmtrm_dividends, thstrm_dividends

In [16]:
data = []
for idx, stock_name in enumerate(stock_name_list[:20]):
  print(idx+1, "/", len(stock_name_list))
  record = [stock_name]
  for year in [2015,2018,2020]:
    lwfr_dividends, frmtrm_dividends, thstrm_dividends = find_dividends(stock_name, year)

    if year != 2020:
      record += [lwfr_dividends, frmtrm_dividends, thstrm_dividends]
    else:
      record += [frmtrm_dividends, thstrm_dividends]
    
    data.append(record)
    time.sleep(0.5)

1 / 2370
2 / 2370
3 / 2370
4 / 2370
5 / 2370
6 / 2370
7 / 2370
8 / 2370
9 / 2370
10 / 2370
11 / 2370
12 / 2370
13 / 2370
14 / 2370
15 / 2370
16 / 2370
17 / 2370
18 / 2370
19 / 2370
20 / 2370


In [17]:
data = pd.DataFrame(data,columns=["stock_name","2015","2016","2017","2018","2019","2020","2021","2022"])
data.to_csv("배당목록.csv")

In [18]:
3

3

# 주요 재무지표 수집 및 가공

In [19]:
result = dart.finstate("LG전자", 2020,"11011")
result = result.loc[result['fs_nm'] == "연결재무제표"]
result = result.loc[result['account_nm'].isin(['당기순이익','영업이익','매출액'])]

result = result.loc[:,["account_nm","fs_nm","sj_nm","thstrm_dt","thstrm_amount"]]

display(result)


,account_nm,fs_nm,sj_nm,thstrm_dt,thstrm_amount
9,매출액,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"63,262,046,000,000"
10,영업이익,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"3,194,987,000,000"
12,당기순이익,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"2,063,790,000,000"


In [20]:
stock_list = stock_list.loc[stock_list['Market'].isin(["KOSPI","KOSDAQ"])]
stock_name_list = stock_list.loc[stock_list['Region'].notnull(),'Name'].tolist()


In [22]:
def find_financial_indicators(stock_name, year, indicators):
    report = dart.finstate(stock_name, year)
    if report is None:
        data = [[stock_name, year] + [np.nan] * len(indicators)]
        data = [[stock_name, year-1] + [np.nan] * len(indicators)]
        data = [[stock_name, year-2] + [np.nan] * len(indicators)]
        
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)
    
    else:
        report = report[report['account_nm'].isin(indicators)]
        
        if sum(report['fs_nm'] == "연결재무제표") >0:
            report = report.loc[report['fs_nm'] == "연결재무제표"]
        
        else:
            report = report.loc[report['fs_nm'] == "연결재무제표"]
        
        data = []
        for y,c in zip([year, year-1, year-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [stock_name, y]
            for indic in indicators:
                if sum(report['account_nm'] == indic) > 0:
                    value = report.loc[report['account_nm'] == indic, c].iloc[0]
                else:
                    value = np.nan
                record.append(value)
                
            data.append(record)
        return pd.DataFrame(data,columns = ["기업", "연도"] + indicators)

In [23]:
indicators = ["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"]
display(find_financial_indicators("삼성전자", 2020, indicators))

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000"
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000"
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000"


In [26]:
data = pd.DataFrame()

for idx, stock_name in enumerate(stock_name_list[:20]):
    print(idx+1, "/", len(stock_name_list))
    for year in [2015,2018,2020]:
        try:
            result = find_financial_indicators(stock_name, year, indicators)
        except:
            pass
        data = pd.concat([data, result], axis=0, ignore_index=True)
        time.sleep(0.5)

1 / 2370
2 / 2370
3 / 2370
4 / 2370
5 / 2370
6 / 2370
7 / 2370
8 / 2370
9 / 2370
10 / 2370
11 / 2370
12 / 2370
13 / 2370
14 / 2370
15 / 2370
16 / 2370
17 / 2370
18 / 2370
19 / 2370
20 / 2370


In [27]:
# 중복 행 제거
data.drop_duplicates(inplace = True)

In [28]:
def str_to_float(value):
    if type(value) == float:
        return value
    elif value == "-":
        return 0
    else:
        return float(value.replace(',',''))

for indc in indicators:
    data[indc] = data[indc].apply(str_to_float)

In [29]:
#부채 비율
data['부채비율'] = data['자본총계'] / data['자산총계'] * 100
display(data['부채비율'].head())

0    56.323967
1    60.288336
2    70.612320
3    62.099593
4    54.331315
Name: 부채비율, dtype: float64

In [30]:
# 영업이익 증가율
data.sort_values(by=['기업','연도'], inplace=True)
data['영업이익증가율'] = data['영업이익'].diff() / data['영업이익']
data.loc[data['연도'] == 2013, '영업이익증가율'] = np.nan

In [31]:
# 매출액
data.sort_values(by=['기업','연도'], inplace=True)
data['매출액증가율'] = data['매출액'].diff() / data['매출액']
data.loc[data['연도'] == 2013, '매출액증가율'] = np.nan

In [32]:
# 당기순이익
data.sort_values(by=['기업','연도'], inplace=True)
data['당기순이익증가율'] = data['당기순이익'].diff() / data['당기순이익']
data.loc[data['연도'] == 2013, '당기순이익증가율'] = np.nan

In [33]:
# 영업이익 / 매출액 / 당기순이익 상태
data['매출액_상태'] = np.nan

data.loc[(data["매출액"].iloc[1:] > 0) & (data["매출액"].iloc[:-1] > 0), "매출액_상태"] = "흑자지속"
data.loc[(data["매출액"].iloc[1:] <= 0) & (data["매출액"].iloc[:-1] <= 0), "매출액_상태"] = "적자지속"
data.loc[(data["매출액"].iloc[1:] > 0) & (data["매출액"].iloc[:-1] <= 0), "매출액_상태"] = "흑자전환"
data.loc[(data["매출액"].iloc[1:] <= 0) & (data["매출액"].iloc[:-1] > 0), "매출액_상태"] = "적자전환"

data.loc[data['연도'] == 2013, "매출액_상태"] = np.nan


In [35]:
# ROA

data ["ROA"] = data['당기순이익'] / data['자산총계']

# ROE
average_equity = data['자본총계'].rolling(2).mean()
data['ROE'] = data['당기순이익'] / average_equity


In [36]:
data.to_csv("주요재무지표.csv", index=False)

# 전체 재무제표 수집

In [38]:
stock_list = fdr.StockListing("KRX").dropna()

In [39]:
# 숫자로 모두 변환
def convert_str_to_float(value):
    if type(value) == float: # nan의 자료형은 float임
        return value
    elif value == '-': # -로 되어 있으면 0으로 변환
        return 0
    else:
        return float(value.replace(',', ''))

In [40]:
def extract_info_and_save(dart, path, year, name, code, report_type, report_type_code):
    report = dart.finstate(code, year, report_type_code)
    if report is None:
        pass
    else:
        submission_date = report['rcept_no'].iloc[0][:8]
        report = report[['fs_nm', 'account_nm', 'thstrm_dt', 'thstrm_amount', 'sj_nm']]
        report.rename({"fs_nm":"개별/연결",
                       "account_nm":"계정명",
                       "sj_nm":"재무제표명",
                       "thstrm_dt": "당기일자",
                       "thstrm_amount":"금액"}, axis = 1, inplace = True)
        report['금액'] = report['금액'].apply(convert_str_to_float)
        
        if name not in os.listdir(path):
            os.mkdir(path + "/" + name)
            
        report.to_csv(path + "/{}/{}_{}년_{}.csv".format(name, submission_date, year, report_type),
                      index = False, encoding = "euc-kr")

In [44]:
dart = OpenDartReader(my_api)
for code, name in stock_list[:2][['Symbol', 'Name']].values:
    for year in range(2016, 2022):
        for report_type, report_type_code in zip(["1분기보고서", "반기보고서", "3분기보고서", "사업보고서"],
                                                  ["11013", "11012", "11014", "11011"]):
            print(name, year, report_type)
            while True:
                try:
                    extract_info_and_save(dart = dart,
                                          path = r".",
                                          year = year,
                                          name = name,
                                          code = code,
                                          report_type = report_type,
                                          report_type_code = report_type_code)
                    time.sleep(0.5)
                    break
                    
                except:
                    time.sleep(60)

3S 2016 1분기보고서
3S 2016 반기보고서
3S 2016 3분기보고서
3S 2016 사업보고서
3S 2017 1분기보고서
3S 2017 반기보고서
3S 2017 3분기보고서
3S 2017 사업보고서
3S 2018 1분기보고서
3S 2018 반기보고서
3S 2018 3분기보고서
3S 2018 사업보고서
3S 2019 1분기보고서
3S 2019 반기보고서
3S 2019 3분기보고서
3S 2019 사업보고서
3S 2020 1분기보고서
3S 2020 반기보고서
3S 2020 3분기보고서
3S 2020 사업보고서
3S 2021 1분기보고서
3S 2021 반기보고서
3S 2021 3분기보고서
3S 2021 사업보고서
AJ네트웍스 2016 1분기보고서
AJ네트웍스 2016 반기보고서
AJ네트웍스 2016 3분기보고서
AJ네트웍스 2016 사업보고서
AJ네트웍스 2017 1분기보고서
AJ네트웍스 2017 반기보고서
AJ네트웍스 2017 3분기보고서
AJ네트웍스 2017 사업보고서
AJ네트웍스 2018 1분기보고서
AJ네트웍스 2018 반기보고서
AJ네트웍스 2018 3분기보고서
AJ네트웍스 2018 사업보고서
AJ네트웍스 2019 1분기보고서
AJ네트웍스 2019 반기보고서
AJ네트웍스 2019 3분기보고서
AJ네트웍스 2019 사업보고서
AJ네트웍스 2020 1분기보고서
AJ네트웍스 2020 반기보고서
AJ네트웍스 2020 3분기보고서
AJ네트웍스 2020 사업보고서
AJ네트웍스 2021 1분기보고서
AJ네트웍스 2021 반기보고서
AJ네트웍스 2021 3분기보고서
AJ네트웍스 2021 사업보고서


# References

In [ ]:
'''
https://docs.python-requests.org/en/master/
''' 